In [86]:
import pandas as pd
import numpy as np
import datetime as dt
from importlib import reload
import TSManager as ts
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
plt.style.use('ggplot')
%matplotlib inline

In [2]:
reload(ts)
mgr = ts.TSManager()
mgr.load()

Loading DP
Loading PE
Loading BM
Loading CAPE
Loading BY
Loading DEF
Loading PCAPrice
Loading CPI
Loading NOS
Loading PCR
Loading OIL
Loading SI
Loading BDI
Loading CAY
Loading IC
Loading TERM
Loading VRP
Loading SPX


In [73]:
df = mgr.data
df.head()

,DP,PE,BM,CAPE,BY,DEF,PCAPrice,CPI,NOS,PCR,OIL,SI,BDI,CAY,IC,TERM,VRP,SPX
AsOfDate,,,,,,,,,,,,,,,,,,
1990-06-08,0.063979,7.524271,NaN,8.978524,1.007418,0.96,3.616905,0.046737,0.016941,-1.622923,NaN,NaN,1278.0,-0.013576,NaN,0.50,7.197727,-0.012226
1990-06-11,0.063463,7.597792,NaN,9.051612,1.009642,0.96,3.583706,0.046737,0.016941,-1.615366,NaN,NaN,1264.0,-0.013576,NaN,0.49,5.607431,0.008140
1990-06-12,0.062662,7.699009,NaN,9.167251,1.009580,0.96,3.532720,0.046737,0.016941,-1.618868,NaN,NaN,1254.0,-0.013576,NaN,0.49,6.403252,0.012775
1990-06-13,0.062894,7.674771,NaN,9.133460,1.000056,0.96,3.547245,0.046737,0.016941,-1.625316,NaN,NaN,1243.0,-0.013576,NaN,0.47,7.793147,-0.003686
1990-06-14,0.063241,7.636994,NaN,9.083400,0.997689,0.96,3.569002,0.046737,0.016941,-1.620454,NaN,NaN,1232.0,-0.013576,NaN,0.46,7.665179,-0.005481


In [133]:
class CorrScreenRegressions:

    def __init__(self,threshold=0.1,R_130=None):
        self.threshold = threshold
        self.R_130 = R_130

    def fit(self,X,y):

        X = X.T

        # screen variables: measure pairwise correlation between X(t) and y(t+130)
        if self.R_130 is None:
            self.R_130 = [y[i:i+130].sum() for i in range(X.shape[1]-130)]
              
        screened_ft = [i for i in range(X.shape[0]) if abs(np.corrcoef(np.array([X[i,:-130],self.R_130]))[0,1])>self.threshold]
        self.screened_ft = screened_ft

        # fit
        self.reg = LinearRegression().fit(X[screened_ft,:].T, y)

    def predict(self,X):
        X = X.T
        return self.reg.predict(X[self.screened_ft,:].T)

    def return_screened_ft(self):
        return self.screened_ft
    
class CorrScreenPredictor:
    
    def __init__(self,data,threshold=0.1):
        self.threshold = threshold
        self.data = data
        self.X = np.array(data[['DP','PE','BM','CAPE','BY','DEF',
                       'PCAPrice','CPI','NOS','PCR','OIL','SI',
                      'BDI','CAY','IC','TERM','VRP']])
        self.y = np.array(data[['SPX']])
    
    def predict(self,nb_periods):

        p = []
        sft = []

        for t in range(nb_periods):

            X = self.X[20*t:(2520+20*t),:]
            next_X = self.X[(2520+20*t):(2540+20*t),:]
            y = self.y[20*t:(2520+20*t)]

            CSR = CorrScreenRegressions(self.threshold)
            CSR.fit(X,y)
            sft.append(CSR.return_screened_ft())

            for el in CSR.predict(next_X):
                p.append(el)

        return p,self.y[2520:(2540+20*(nb_periods-1))],sft
    
    def plot_predict(self,nb_periods):
        
        p,ytrue,sft = self.predict(nb_periods)
        
        plt.figure(figsize=(15,5))
        plt.plot(self.data.index[2520:(2520+nbp*20)],np.cumsum(p),label='screened ft')
        plt.plot(self.data.index[2520:(2520+nbp*20)],np.cumsum(ytrue),label='true')
        plt.title('linear regression for predicting returns')
        plt.legend()
        plt.show()
    
    def plot_screened_ft(self,nb_periods):
        
        p,ytrue,sft = self.predict(nb_periods)
        msft = []
        for i in range(nb_periods):
            for k in range(20):
                msft.append(len(sft[i]))
        plt.figure(figsize=(15,5))
        plt.plot(self.data.index[2520:(2520+nbp*20)],msft,label='screened ft')
        plt.ylim(0,20)
        plt.title('Number of features used')
        plt.legend()
        plt.show()
        
    
    

In [134]:
# test predictions

nbp = 180 # number of periods to be used
CSP = CorrScreenPredictor(df,0)
CSP.plot_predict(nbp)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# plot total ft used per period

CSP.plot_screened_ft(nbp)